In [ ]:
import torch as tc 
import torch.nn as nn  
import numpy as np
from tqdm import tqdm
from torch.cuda.amp import autocast
import cv2
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
!python -m pip install --no-index --find-links=/kaggle/input/pip-download-for-segmentation-models-pytorch segmentation-models-pytorch
import segmentation_models_pytorch as smp
# !python -m pip install /kaggle/input/ttach003/ttach-0.0.3-py3-none-any.whl
# import ttach as tta
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from dotenv import load_dotenv

In [ ]:
%pip install --no-index --find-links /kaggle/input/install-nvidia-tensorrt/nvidia/ tensorrt
# import os, tensorrt
# os.environ["LD_LIBRARY_PATH"] = "/opt/conda/lib/python3.10/site-packages/tensorrt/"
%pip install --no-index --find-links /kaggle/input/install-nvidia-tensorrt/torch2trt/ torch2trt
%pip install --no-index --find-links /kaggle/input/install-nvidia-tensorrt/torch2trt/ nvidia-pyindex
%pip install --no-index --find-links /kaggle/input/install-nvidia-tensorrt/torch2trt/ onnx-graphsurgeon
%pip install --no-index --find-links /kaggle/input/install-nvidia-tensorrt/torch2trt/ onnxruntime

In [ ]:
model1_path_i = 0    # 12 #9 # 7 #5 #in_chans_1__25     3 in_chans_1__20 2 
#model_path_i9 = 12  # 9 #11
model2_path_i = 4

class CFG:
# ============== model CFG =============
    model_name = 'Unet'
    backbone1 = 'se_resnext50_32x4d'
    in_chans1 = 1 #5 # 65
    
    backbone2 = 'timm-efficientnet-b2'
    in_chans2 = 1 #5 # 65
    
    ensemble_weight = 0.9
    #============== tta cfg =============
    num_rot90=4
    num_flip=0
    #============== _ CFG =============
    image_size = 1024 #512
    input_size= 1024 #512
    tile_size = image_size
    stride = tile_size // 4
    drop_egde_pixel= 0 # 16 #32
    
    target_size = 1
    chopping_percentile=1e-3
    # ============== fold =============
    valid_id = 1
    batch=16 #128

    th_percentile_i = 4
    th_percentile_list = [0.0013099, 0.0013499, 0.0013999, 0.0014499,   0.00143 ]

    th_percentile = th_percentile_list[th_percentile_i] #0.00143 #0.00145 #0.00146 #0.00149 #0.00145 # 0.0014 #0.00175 #0.0021
    
    #axis_w = [0.3353333 ,0.3323333,0.3323333 ]
    model_path=[
        "/kaggle/input/sennet-hoa-trt-models/model9.pth",
        "/kaggle/input/sennet-hoa-trt-models/model18.pth",
        "/kaggle/input/sennet-hoa-tensorrt/model19.pth",
        "/kaggle/input/sennet-hoa-trt-models/model22.pth",
        "/kaggle/input/sennet-hoa-trt-models/model27.pth",
    ]

In [ ]:
class UnetModel(nn.Module):
    def __init__(self, backbone, in_chans, target_size, weight):
        super().__init__()
        self.model = smp.Unet( #FPN
            encoder_name=backbone, 
            encoder_weights=weight,
            in_channels=in_chans,
            classes=target_size,
            activation=None,
        )
    def forward(self, x):
        output = self.model(x)
        return output[:, 0]
class UnetPlusPlusModel(nn.Module):
    def __init__(self, backbone, in_chans, target_size, weight):
        super().__init__()
        self.model = smp.UnetPlusPlus( #FPN
            encoder_name=backbone, 
            encoder_weights=weight,
            in_channels=in_chans,
            classes=target_size,
            activation=None,
        )
    def forward(self, x):
        output = self.model(x)
        return output[:, 0]
class FPNModel(nn.Module):
    def __init__(self, backbone, in_chans, target_size, weight):
        super().__init__()
        self.model = smp.FPN( #FPN
            encoder_name=backbone, 
            encoder_weights=weight,
            in_channels=in_chans,
            classes=target_size,
            activation=None,
        )
    def forward(self, x):
        output = self.model(x)
        return output[:, 0]

class EnsembleModel(nn.Module):
    def __init__(self, CFG, weight1=None, weight2=None):
        super().__init__()
        self.CFG = CFG        
        self.model1 = UnetModel(backbone=CFG.backbone1,
                                in_chans=CFG.in_chans1,
                                target_size=CFG.target_size,
                                weight=weight1)
        self.model2 = UnetModel(backbone=CFG.backbone2,
                                in_chans=CFG.in_chans2,
                                target_size=CFG.target_size,
                                weight=weight2)
        self.batch=CFG.batch
        self.ensemble_weight = CFG.ensemble_weight
    
    def forward(self,x:tc.Tensor):
        #x.shape=(batch,c,h,w)
        x=x.to(tc.float32)
        x=norm_with_clip(x.reshape(-1,*x.shape[2:])).reshape(x.shape)
        
        if CFG.input_size!=CFG.image_size:
            x=nn.functional.interpolate(x,size=(CFG.input_size,CFG.input_size),mode='bilinear',align_corners=True)
        
        shape=x.shape
        x=[tc.rot90(x,k=i,dims=(-2,-1)) for i in range(CFG.num_rot90)]
        x.extend([tc.flip(x[i], dims=(-1,)) for i in range(CFG.num_flip)])

        x=tc.cat(x,dim=0)
        with autocast():
            with tc.no_grad():
                x1=[self.model1(x[i*self.batch:(i+1)*self.batch]) for i in range((x.shape[0]-1)//self.batch+1)]
                # batch=64,64...48
                x1=tc.cat(x1,dim=0)
                
                x2=[self.model2(x[i*self.batch:(i+1)*self.batch]) for i in range((x.shape[0]-1)//self.batch+1)]
                # batch=64,64...48
                x2=tc.cat(x2,dim=0)
        x1=x1.sigmoid()
        x1=x1.reshape(CFG.num_rot90+CFG.num_flip,shape[0],*shape[2:])
        x1_=[tc.rot90(x1[i],k=-i,dims=(-2,-1)) for i in range(CFG.num_rot90)]
        x1_.extend([tc.rot90(tc.flip(x1[i+4], dims=(-1,)),k=-i,dims=(-2,-1)) for i in range(CFG.num_flip)])
        x1=tc.stack(x1_,dim=0).mean(0)
        
        
        x2=x2.sigmoid()
        x2=x2.reshape(CFG.num_rot90+CFG.num_flip,shape[0],*shape[2:])
        x2_=[tc.rot90(x2[i],k=-i,dims=(-2,-1)) for i in range(CFG.num_rot90)]
        x2_.extend([tc.rot90(tc.flip(x2[i+4], dims=(-1,)),k=-i,dims=(-2,-1)) for i in range(CFG.num_flip)])
        x2=tc.stack(x2_,dim=0).mean(0)
        
        x = x1*self.ensemble_weight + x2*(1-self.ensemble_weight)
        if CFG.input_size!=CFG.image_size:
            x=nn.functional.interpolate(x[None],size=(CFG.image_size,CFG.image_size),mode='bilinear',align_corners=True)[0]
        return x


def build_model(weight1=None, weight2=None):
    load_dotenv()

    print('model_name', CFG.model_name)
    print('backbone1', CFG.backbone1)
    print('backbone2', CFG.backbone2)


    model = EnsembleModel(CFG, weight1, weight2)

    return model.cuda()

In [ ]:
def to_1024(img , image_size = 1024):
    if image_size > img.shape[1]:
       img = np.rot90(img)
       start1 = (CFG.image_size - img.shape[0])//2 
       top =     img[0                    : start1,   0: img.shape[1] ]
       bottom  = img[img.shape[0] -start1 : img.shape[0],   0 : img.shape[1] ]
       img_result = np.concatenate((top,img,bottom ),axis=0)
       img_result = np.rot90(img_result)
       img_result = np.rot90(img_result)
       img_result = np.rot90(img_result)
    else :
       img_result = img
    return img_result

def to_1024_no_rot(img, image_size = 1024):
    if image_size > img.shape[0]:  
       start1 = ( image_size - img.shape[0])//2
       top =     img[0                    : start1,   0: img.shape[1] ]
       bottom  = img[img.shape[0] -start1 : img.shape[0],   0 : img.shape[1] ]
       img_result = np.concatenate((top,img,bottom ),axis=0)
    else: 
       img_result = img
    return img_result

def to_1024_1024(img  , image_size = 1024 ):
     img_result = to_1024(img, image_size )
     return img_result
    
def to_original ( im_after, img, image_size = 1024 ):
    top_ = 0
    left_ = 0
    if (im_after.shape[0] > img.shape[0]):
             top_  = ( image_size - img.shape[0])//2 
    if    (im_after.shape[1] > img.shape[1]) :
             left_  = ( image_size - img.shape[1])//2  
    if (top_>0)or (left_>0) :
             img_result = im_after[top_  : img.shape[0] + top_,   left_: img.shape[1] + left_ ]
    else:
             img_result = im_after
    return img_result  

In [ ]:
def rle_encode(mask):
    pixel = mask.flatten()
    pixel = np.concatenate([[0], pixel, [0]])
    run = np.where(pixel[1:] != pixel[:-1])[0] + 1
    run[1::2] -= run[::2]
    rle = ' '.join(str(r) for r in run)
    if rle == '':
        rle = '1 0'
    return rle

def min_max_normalization(x:tc.Tensor)->tc.Tensor:
    """input.shape=(batch,f1,...)"""
    shape=x.shape
    if x.ndim>2:
        x=x.reshape(x.shape[0],-1)
    
    min_=x.min(dim=-1,keepdim=True)[0]
    max_=x.max(dim=-1,keepdim=True)[0]
    if min_.mean()==0 and max_.mean()==1:
        return x.reshape(shape)
    
    x=(x-min_)/(max_-min_+1e-9)
    return x.reshape(shape)

def norm_with_clip(x:tc.Tensor,smooth=1e-5):
    dim=list(range(1,x.ndim))
    mean=x.mean(dim=dim,keepdim=True)
    std=x.std(dim=dim,keepdim=True)
    x=(x-mean)/(std+smooth)
    x[x>5]=(x[x>5]-5)*1e-3 +5
    x[x<-3]=(x[x<-3]+3)*1e-3-3
    return x

class Data_loader(Dataset):
    def __init__(self,path,s="/images/"):
        self.paths=glob(path+f"{s}*.tif")
        self.paths.sort()
        self.bool=s=="/labels/"
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self,index):
        img=cv2.imread(self.paths[index],cv2.IMREAD_GRAYSCALE)
        img = to_1024_1024(img , image_size = CFG.image_size )
        
        img=tc.from_numpy(img.copy())
        if self.bool:
            img=img.to(tc.bool)
        else:
            img=img.to(tc.uint8)
        return img

def load_data(path,s):
    data_loader=Data_loader(path,s)
    data_loader=DataLoader(data_loader, batch_size=16, num_workers=2)
    data=[]
    for x in tqdm(data_loader):
        data.append(x)
    x=tc.cat(data,dim=0)
    ########################################################################
    TH=x.reshape(-1).numpy()
    index = -int(len(TH) * CFG.chopping_percentile)
    TH:int = np.partition(TH, index)[index]
    x[x>TH]=int(TH)
    ########################################################################
    TH=x.reshape(-1).numpy()
    index = -int(len(TH) * CFG.chopping_percentile)
    TH:int = np.partition(TH, -index)[-index]
    x[x<TH]=int(TH)
    ########################################################################
    #x=(min_max_normalization(x.to(tc.float16))*255).to(tc.uint8)
    return x

class Pipeline_Dataset(Dataset):
    def __init__(self,x,path):
        self.img_paths  = glob(path+"/images/*")
        self.img_paths.sort()
        self.in_chan = CFG.in_chans1
        z=tc.zeros(self.in_chan//2,*x.shape[1:],dtype=x.dtype)
        self.x=tc.cat((z,x,z),dim=0)
        
    def __len__(self):
        return self.x.shape[0]-self.in_chan+1
    
    def __getitem__(self, index):
        x  = self.x[index:index+self.in_chan]
        return x,index
    
    def get_mark(self,index):
        id=self.img_paths[index].split("/")[-3:]
        id.pop(1)
        id="_".join(id)
        return id[:-4]
    
    def get_marks(self):
        ids=[]
        for index in range(len(self)):
            ids.append(self.get_mark(index))
        return ids

def add_edge(x:tc.Tensor,edge:int):
    #x=(C,H,W)
    #output=(C,H+2*edge,W+2*edge)
    mean_=int(x.to(tc.float32).mean())
    x=tc.cat([x,tc.ones([x.shape[0],edge,x.shape[2]],dtype=x.dtype,device=x.device)*mean_],dim=1)
    x=tc.cat([x,tc.ones([x.shape[0],x.shape[1],edge],dtype=x.dtype,device=x.device)*mean_],dim=2)
    x=tc.cat([tc.ones([x.shape[0],edge,x.shape[2]],dtype=x.dtype,device=x.device)*mean_,x],dim=1)
    x=tc.cat([tc.ones([x.shape[0],x.shape[1],edge],dtype=x.dtype,device=x.device)*mean_,x],dim=2)
    return x

In [ ]:
from torch2trt import TRTModule
import torch
model1_trt = TRTModule()
model2_trt = TRTModule()

model1_trt.load_state_dict(torch.load(CFG.model_path[ model1_path_i ]))
model2_trt.load_state_dict(torch.load(CFG.model_path[ model2_path_i ]))

In [ ]:
model=build_model(   )
model.model1 = model1_trt.cuda()
model.model2 = model2_trt.cuda()

model.eval()
# model=DataParallel(model)


In [ ]:
def get_output(debug=False):
    outputs=[]
    
    if debug:
        paths=["/kaggle/input/blood-vessel-segmentation/train/kidney_2"]
    else:
        paths=glob("/kaggle/input/blood-vessel-segmentation/test/*")
    outputs=[[],[]]
    for path in paths:
        x=load_data(path,"/images/")
        labels=tc.zeros_like(x,dtype=tc.uint8)
        mark=Pipeline_Dataset(x,path).get_marks()
        
        
        for axis in [0,1,2]:
            debug_count=0
            if axis==0:
                x_=x
                labels_=labels
            elif axis==1:
                x_=x.permute(1,2,0)
                labels_=labels.permute(1,2,0)
            elif axis==2:
                x_=x.permute(2,0,1)
                labels_=labels.permute(2,0,1)
            if x.shape[0]==3 and axis!=0:
                break
            dataset=Pipeline_Dataset(x_,path)
            dataloader=DataLoader(dataset,batch_size=1,shuffle=False,num_workers=1)
            shape=dataset.x.shape[-2:]
            x1_list = np.arange(0, shape[0]+CFG.tile_size-CFG.tile_size+1, CFG.stride)
            y1_list = np.arange(0, shape[1]+CFG.tile_size-CFG.tile_size+1, CFG.stride)
            for img,index in tqdm(dataloader):
                #img=(1,C,H,W)
                img=img.to("cuda:0")
                img=add_edge(img[0],CFG.tile_size//2)[None]

                mask_pred = tc.zeros_like(img[:,0],dtype=tc.float32,device=img.device)
                mask_count = tc.zeros_like(img[:,0],dtype=tc.float32,device=img.device)

                indexs=[]
                chip=[]
                for y1 in y1_list:
                    for x1 in x1_list:
                        x2 = x1 + CFG.tile_size
                        y2 = y1 + CFG.tile_size
                        indexs.append([x1+CFG.drop_egde_pixel,x2-CFG.drop_egde_pixel,
                                       y1+CFG.drop_egde_pixel,y2-CFG.drop_egde_pixel])
                        chip.append(img[...,x1:x2,y1:y2])

                y_preds = model.forward(tc.cat(chip)).to(device=0)
                
                
                if CFG.drop_egde_pixel:
                    y_preds=y_preds[...,CFG.drop_egde_pixel:-CFG.drop_egde_pixel,
                                        CFG.drop_egde_pixel:-CFG.drop_egde_pixel]
                for i,(x1,x2,y1,y2) in enumerate(indexs):
                    mask_pred[...,x1:x2, y1:y2] += y_preds[i]
                    mask_count[...,x1:x2, y1:y2] += 1

                mask_pred /= mask_count

                #Rrecover
                mask_pred=mask_pred[...,CFG.tile_size//2:-CFG.tile_size//2,CFG.tile_size//2:-CFG.tile_size//2]
                
                labels_[index]+=(mask_pred[0]*255 /3 ).to(tc.uint8).cpu()
                if debug:
                    debug_count+=1
                    plt.subplot(121)
                    plt.imshow(img[0,CFG.in_chans1//2].cpu().detach().numpy())
                    plt.subplot(122)
                    plt.imshow(mask_pred[0].cpu().detach().numpy())
                    plt.show()
                    if debug_count>3:
                        break
        outputs[0].append(labels)
        outputs[1].extend(mark)
    return outputs

In [ ]:
is_submit=len(glob("/kaggle/input/blood-vessel-segmentation/test/kidney_5/images/*.tif"))!=3
#is_submit=True
output,ids=get_output(not is_submit)


####################################
TH=[x.flatten().numpy() for x in output]
TH=np.concatenate(TH)
index = -int(len(TH) * CFG.th_percentile)
TH:int = np.partition(TH, index)[index]
print(TH)

img=cv2.imread("/kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0001.tif",cv2.IMREAD_GRAYSCALE)

####################################
submission_df=[]
debug_count=0
for index in range(len(ids)):
    id=ids[index]
    i=0
    for x in output:
        if index>=len(x):
            index-=len(x)
            i+=1
        else:
            break
    mask_pred=(output[i][index]>TH).numpy()
    
    mask_pred2 = to_original ( mask_pred, img, image_size = 1024 )
    mask_pred =  mask_pred2.copy()
    
    ####################################
    if not is_submit:
        plt.subplot(121)
        plt.imshow(mask_pred)
        plt.show()
        debug_count+=1
        if debug_count>6:
            break
        
    rle = rle_encode(mask_pred)
    
    submission_df.append(
        pd.DataFrame(data={
            'id'  : id,
            'rle' : rle,
        },index=[0])
    )

submission_df =pd.concat(submission_df)
submission_df.to_csv('submission.csv', index=False)
submission_df.head(6)

In [ ]:
import gc
gc.collect()
tc.cuda.empty_cache()